### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR(pair-wise loss) на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP(list-wise loss) на implicit данных

Мягкий дедлайн 28 Сентября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 5 Октября (Итоговая проверка)

In [1]:
!pip install implicit

     |████████████████████████████████| 1.1MB 2.8MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp36-cp36m-linux_x86_64.whl size=3419421 sha256=3a794817084451a00ef560c7813e3b16259a7256df44c9a848e2b676995a3688
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit


In [2]:
!pip install lightfm

     |████████████████████████████████| 307kB 2.7MB/s 
  Created wheel for lightfm: filename=lightfm-1.15-cp36-cp36m-linux_x86_64.whl size=707631 sha256=231a7134f5c64229e75b2d6513ac35f1606bf20434c0d370fe452b5f8244b7c3
  Stored in directory: /root/.cache/pip/wheels/eb/bb/ac/188385a5da6627956be5d9663928483b36da576149ab5b8f79
Successfully built lightfm


In [3]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp

from lightfm.datasets import fetch_movielens

В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
path = '/content/gdrive/My Drive/Colab Notebooks/'

Mounted at /content/gdrive


In [5]:
ratings = pd.read_csv(path+'RecSysHSE/ml-1m/ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [6]:
movie_info = pd.read_csv(path+'RecSysHSE/ml-1m/movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python')

In [7]:
ratings = ratings.sort_values(by = ['user_id', 'movie_id'])

Explicit данные

In [ ]:
ratings.head(10)

,user_id,movie_id,rating
40,1,1,5
25,1,48,5
39,1,150,5
44,1,260,4
23,1,527,5
49,1,531,4
33,1,588,4
8,1,594,4
10,1,595,5
51,1,608,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [8]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]
implicit_ratings['rating'] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
implicit_ratings.head(10)

,user_id,movie_id,rating
40,1,1,1
25,1,48,1
39,1,150,1
44,1,260,1
23,1,527,1
49,1,531,1
33,1,588,1
8,1,594,1
10,1,595,1
51,1,608,1


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [ ]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [ ]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [ ]:
model.fit(user_item_t_csr)

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [ ]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [46]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [ ]:
get_similars(1, model)

['0    Toy Story (1995)',
 '379    Wyatt Earp (1994)',
 'Series([], )',
 '2330    Santa Claus: The Movie (1985)',
 '3699    Braddock: Missing in Action III (1988)',
 '1454    Daytrippers, The (1996)',
 '3327    Muppet Movie, The (1979)',
 '3579    Abominable Snowman, The (1957)',
 '1795    Sour Grapes (1998)',
 '2641    Blair Witch Project, The (1999)']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [9]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [ ]:
get_user_history(4, implicit_ratings)

['257    Star Wars: Episode IV - A New Hope (1977)',
 '476    Jurassic Park (1993)',
 '1023    Die Hard (1988)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '1180    Raiders of the Lost Ark (1981)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '1196    Alien (1979)',
 '1220    Terminator, The (1984)',
 '1366    Jaws (1975)',
 '1885    Rocky (1976)',
 '1959    Saving Private Ryan (1998)',
 '2297    King Kong (1933)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '2882    Fistful of Dollars, A (1964)',
 '3349    Thelma & Louise (1991)',
 '3399    Hustler, The (1961)',
 '3633    Mad Max (1979)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [10]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [ ]:
get_recommendations(4, model)

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [49]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
class SGD:
    def __init__(self, r = 64, reg_par = 0.01, lr = 0.01, eps = 5e-4, iters = int(5e6)):
        self.r = r
        self.reg_par = reg_par
        self.lr = lr
        self.eps = eps
        self.iters = iters
        
    def fit(self, M):   
        row = np.max(M['user_id'])
        col = np.max(M['movie_id'])
        self.W = np.random.uniform(0, 1/np.sqrt(self.r), (row, self.r))
        self.H = np.random.uniform(0, 1/np.sqrt(self.r), (self.r, col))
        #B = np.random.uniform(0, 1/np.sqrt(self.r), (row, col))
        self.B_u = np.zeros((row, 1))
        self.B_i = np.zeros((1, col))
        self.mu = M['rating'].mean()
        total_len = len(M)
        rmse = np.linalg.norm((self.W@self.H + self.B_u + self.B_i + self.mu)[M['user_id']-1, M['movie_id']-1]
                                                   - M['rating'])/total_len
        t = 0
        while t < self.iters:
            k = np.random.randint(total_len)
            i = M.iloc[k]['user_id']-1
            j = M.iloc[k]['movie_id']-1     
            w, h, b_u, b_i = self.W[i,:], self.H[:,j], self.B_u[i], self.B_i[:,j]
            #b = np.array([B[i[t],j[t]] for t in range(len(i))])
            error = w@h + b_u + b_i + self.mu - M.iloc[k]['rating']

            self.W[i,:] -= self.lr*(error*h.T + self.reg_par*w)
            self.H[:, j] -= self.lr*(error*w.T + self.reg_par*h)
            #B[i, j] -= self.lr*error
            self.B_u[i] -= self.lr*(error + self.reg_par*b_u)
            self.B_i[:,j] -= self.lr*(error + self.reg_par*b_i)
            if t%500000 == 0:
                rmse = np.linalg.norm((self.W@self.H + self.B_u + self.B_i + self.mu)[M['user_id']-1, M['movie_id']-1]
                                                   - M['rating'])/total_len
                if rmse < self.eps:
                    break
                print("rmse", rmse)
            t+=1
        return self.W, self.H, self.B_u, self.B_i, self.mu 
    
    def similar_items(self, item_id):
        item_id-=1
        nbrs = NearestNeighbors(n_neighbors=6, algorithm = 'kd_tree' ).fit(self.H.T)
        distances, indices = nbrs.kneighbors((self.H.T)[item_id].reshape(1,-1))
        return list(zip(indices[0] + 1, distances[0]))
    
    def recommend(self, user_id, M, movie_info):
        user_movie_matrix = self.W@self.H + self.B_u + self.B_i + self.mu
        user_row = user_movie_matrix[user_id - 1]
        watched_movies = M[M['user_id'] == user_id]['movie_id']
        unwatched_movies = [i for i in movie_info['movie_id'] if i not in watched_movies]
        unwatched_movies.sort(key = lambda x: user_row[x-1], reverse = True)
        return list(zip(unwatched_movies[:10], user_row[unwatched_movies[:10]]))

In [47]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, ratings, movie_info)]

In [ ]:
model = SGD()
W, H, B_u, B_i, mu = model.fit(ratings)

rmse 0.001145023688262441
rmse 0.0009305519983153932
rmse 0.0009128954555596631
rmse 0.0009044049328397242
rmse 0.000898116603460106
rmse 0.0008919042464388948
rmse 0.0008844346384120224
rmse 0.0008756991067001193
rmse 0.0008654110640417119
rmse 0.0008539762637158209


In [ ]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 '1949    Bambi (1942)',
 '2011    Lady and the Tramp (1955)',
 '584    Aladdin (1992)',
 '591    Beauty and the Beast (1991)']

In [ ]:
get_recommendations(4, model)

['847    Godfather, The (1972)',
 '2836    Sanjuro (1962)',
 '1189    To Kill a Mockingbird (1962)',
 '910    Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)',
 '892    Rear Window (1954)',
 '1950    Seven Samurai (The Magnificent Seven) (Shichin...',
 '1162    Paths of Glory (1957)',
 '735    Close Shave, A (1995)',
 '315    Shawshank Redemption, The (1994)',
 '901    Maltese Falcon, The (1941)']

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [ ]:
class ALS:
    def __init__(self, r = 64, reg_par = 0.001, lr = 1e-3, eps = 1e-3, iters = 100):
        self.r = r
        self.reg_par = reg_par
        self.lr = lr
        self.eps = eps
        self.iters = iters
        self.movie_info = movie_info
        
    def fit(self, M):   
        row = np.max(M['user_id'])
        col = np.max(M['movie_id'])
        self.W = np.random.uniform(0, 1/np.sqrt(self.r), (row, self.r))
        self.H = np.random.uniform(0, 1/np.sqrt(self.r), (self.r, col))
        total_len = len(M)
        rmse = np.linalg.norm((self.W@self.H)[M['user_id']-1, M['movie_id']-1]
                                                   - M['rating'])/total_len
        t = 0
        while t < self.iters:
            WH = self.W@self.H
            diff = WH
            diff[M['user_id']-1, M['movie_id']-1] = WH[M['user_id']-1, M['movie_id']-1]- M['rating']
            
            if t % 2 == 0:
                self.W = self.W - self.lr * (diff @ self.H.T + self.reg_par * self.W)
            else:
                self.H = self.H - self.lr * (self.W.T @diff  + self.reg_par * self.H)
                
            if t%10 == 0:
                rmse = np.linalg.norm((self.W@self.H)[M['user_id']-1, M['movie_id']-1]
                                                   - M['rating'])/total_len
                print("rmse", rmse)
                
            if rmse < self.eps:
                break
            t+=1
        
        return self.W, self.H  
    
    def similar_items(self,item_id):
        item_id-=1
        nbrs = NearestNeighbors(n_neighbors=10).fit(self.H.T)
        distances, indices = nbrs.kneighbors((self.H.T)[item_id].reshape(1,-1))
        return list(zip(indices[0] + 1, distances[0]))
    
    def recommend(self, user_id, M, movie_info):
        user_movie_matrix = self.W@self.H
        user_row = user_movie_matrix[user_id - 1]
        watched_movies = M[M['user_id'] == user_id]['movie_id']
        unwatched_movies = [i for i in movie_info['movie_id'] if i not in watched_movies]
        unwatched_movies.sort(key = lambda x: user_row[x-1], reverse = True)
        return list(zip(unwatched_movies[:10], user_row[unwatched_movies[:10]]))
    

In [ ]:
model = ALS()
W, H = model.fit(implicit_ratings)

rmse 0.0012476559992730177
rmse 0.0011873506361842945
rmse 0.001129319010494593
rmse 0.0010986231303928575
rmse 0.0010862891137024039
rmse 0.001076605644001585
rmse 0.0010617294794914153
rmse 0.001042069134595005
rmse 0.0010228374209121562
rmse 0.001005967096743313


In [ ]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 '1245    Groundhog Day (1993)',
 "2286    Bug's Life, A (1998)",
 '584    Aladdin (1992)',
 '33    Babe (1995)',
 '352    Forrest Gump (1994)',
 '2327    Shakespeare in Love (1998)',
 '360    Lion King, The (1994)',
 "1854    There's Something About Mary (1998)"]

In [ ]:
get_recommendations(4, model)

['257    Star Wars: Episode IV - A New Hope (1977)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '1180    Raiders of the Lost Ark (1981)',
 '1192    Star Wars: Episode VI - Return of the Jedi (1983)',
 '1220    Terminator, The (1984)',
 '1196    Alien (1979)',
 '1182    Aliens (1986)',
 '585    Terminator 2: Judgment Day (1991)',
 '2502    Matrix, The (1999)',
 '1959    Saving Private Ryan (1998)']

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

In [43]:
class BPR:
    def __init__(self, r = 128, reg_par = 1e-4, lr = 1e-2, eps = 1e-6, iters = int(10)):
        self.r = r
        self.reg_par = reg_par
        self.lr = lr
        self.eps = eps
        self.iters = iters
        self.movie_info = movie_info


    def fit(self, M, movie_info): 
        
        def partial_BPR(x_uij, partial_x):
            exp_x = np.exp(-x_uij)
            return exp_x / (1 + exp_x) * partial_x
        
        row = np.max(M['user_id'])
        col = np.max(M['movie_id'])
        self.W = np.random.uniform(0, 1/np.sqrt(self.r), (row, self.r))
        self.H = np.random.uniform(0, 1/np.sqrt(self.r), (self.r, col))
        total_len = len(M)
        rmse = np.linalg.norm((self.W@self.H)[M['user_id']-1, M['movie_id']-1]
                                                   - M['rating'])/total_len
        
        watched_movies = {}
        unwatched_movies = {}
        unique_users = M["user_id"].unique()-1 
        
        for u in range(1, row + 1):
            watched_movies[u] = np.array(M[M['user_id'] == u]['movie_id'])
            unwatched_movies[u] = np.array([i for i in movie_info['movie_id'] if i not in watched_movies[u]])
        print("end of pre-count")
        
        t = 0
        while t < self.iters:    
            total_loss = 0
            for u in unique_users:
                for i in watched_movies[u+1]:
                    i-=1

                    neg = np.random.choice(unwatched_movies[u+1], 20)-1
         
                    WH_i = self.W[u]@self.H[:,i]
                    WH_j = self.W[u]@self.H[:,neg]
                    x_uij = WH_i - WH_j

                    self.W[u] += self.lr * (partial_BPR(x_uij, self.H[:,i].reshape(-1,1) - self.H[:,neg]).sum(axis=1)  - self.reg_par * self.W[u])
                    self.H[:,i] += self.lr * (partial_BPR(x_uij, self.W[u].reshape(-1,1)).sum(axis=1)  - self.reg_par * self.H[:,i])
                    self.H[:,neg] += self.lr * (partial_BPR(x_uij, -self.W[u].reshape(-1,1)) - self.reg_par * self.H[:,neg]) 

                    total_loss+=np.array(np.log(1 + np.exp(-x_uij))/len(neg)).sum()
            t+=1
            
            if t%1 == 0:
                print(f"step: {t}, loss: {total_loss/len(unique_users)}")

                if total_loss/len(unique_users) < self.eps:
                    break
                
        
        return self.W, self.H  
    
    def similar_items(self, item_id):
        item_id-=1
        nbrs = NearestNeighbors(n_neighbors=10, algorithm = 'kd_tree' ).fit(self.H.T)
        distances, indices = nbrs.kneighbors(((self.H.T)[item_id].reshape(1,-1))
        return list(zip(indices[0] + 1, distances[0]))
    
    def recommend(self, user_id, M, movie_info):
        user_movie_matrix = self.W@self.H
        user_row = user_movie_matrix[user_id - 1]
        watched_movies = M[M['user_id'] == user_id]['movie_id']
        unwatched_movies = [i for i in movie_info['movie_id'] if i not in watched_movies]
        unwatched_movies.sort(key = lambda x: user_row[x-1], reverse = True)
        return list(zip(unwatched_movies[:10], user_row[unwatched_movies[:10]]))
    

In [44]:
model = BPR()
W, H = model.fit(implicit_ratings, movie_info)

end of pre-count
step: 1, loss: 23.833533221579742
step: 2, loss: 16.245393348301516
step: 3, loss: 12.587442696972163
step: 4, loss: 9.703274261772291
step: 5, loss: 7.495840691940639
step: 6, loss: 5.957606344649264
step: 7, loss: 4.937902091957213
step: 8, loss: 4.2566089809862815
step: 9, loss: 3.781963569411705
step: 10, loss: 3.419116622202569


In [50]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '2252    Pleasantville (1998)',
 '584    Aladdin (1992)',
 '2225    Antz (1998)',
 '2012    Little Mermaid, The (1989)',
 '1382    Beavis and Butt-head Do America (1996)',
 "1854    There's Something About Mary (1998)",
 '1595    Full Monty, The (1997)']

In [51]:
get_recommendations(4, model)

['257    Star Wars: Episode IV - A New Hope (1977)',
 '847    Godfather, The (1972)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '1180    Raiders of the Lost Ark (1981)',
 '1220    Terminator, The (1984)',
 '585    Terminator 2: Judgment Day (1991)',
 '1196    Alien (1979)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '1023    Die Hard (1988)',
 '108    Braveheart (1995)']

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных

In [52]:
class WARP:
    def __init__(self, r = 64, reg_par = 0.001, lr = 1e-2, eps = 1e-3, iters = int(10)):
        self.r = r
        self.reg_par = reg_par
        self.lr = lr
        self.eps = eps
        self.iters = iters
        self.movie_info = movie_info
            

    def fit(self, M, movie_info):     
        row = np.max(M['user_id'])
        col = np.max(M['movie_id'])
        self.W = np.random.uniform(0, 1/np.sqrt(self.r), (row, self.r))
        self.H = np.random.uniform(0, 1/np.sqrt(self.r), (self.r, col))
        self.B_i = np.zeros((1, col))
        total_len = len(M)
        
        watched_movies = {}
        unwatched_movies = {}
        unique_users = M["user_id"].unique()-1 
        unique_items = M["movie_id"].unique()
        
        for u in range(1, row + 1):
            watched_movies[u] = np.array(M[M['user_id'] == u]['movie_id'])
            unwatched_movies[u] = np.array([i for i in unique_items if i not in watched_movies[u]])
        print("end of pre-count")
        
        t = 0
        while t < self.iters:
            total_loss = 0
            for u in unique_users:
                unwatched_movies_for_u = unwatched_movies[u+1] - 1
                for i in watched_movies[u+1]:
                    i-=1
                    WH = self.W[u]@self.H[:, i] + self.B_i[:,i]
                    count = 0
                    for j in np.random.permutation(unwatched_movies_for_u)[:20]:
                        count  += 1
                        
                        if self.W[u]@self.H[:, j]+self.B_i[:,j] + 1 > WH:
                            self.W[u] -= self.lr * (np.log(len(unwatched_movies_for_u[:20]) / count ) * (self.H[:, j] - self.H[:, i]) + \
                                                    self.reg_par*self.W[u])
                            self.H[:, i] += self.lr * (np.log(len(unwatched_movies_for_u[:20]) / count ) * self.W[u] - \
                                                       self.reg_par * self.H[:, i])
                            self.H[:, j] -= self.lr * (np.log(len(unwatched_movies_for_u[:20]) / count ) * self.W[u] + \
                                                       self.reg_par * self.H[:, j])
                            
                            self.B_i[:,i] += self.lr * (np.log(len(unwatched_movies_for_u[:20]) / count ) - \
                                                       self.reg_par * self.B_i[:,i])
                            self.B_i[:,j] -= self.lr * (np.log(len(unwatched_movies_for_u[:20]) / count ) + \
                                                       self.reg_par * self.B_i[:,j])
                            total_loss+=np.log(len(unwatched_movies_for_u[:20]) / count ) 
                            break            

            if t%1 == 0:
                print(f"step: {t}, loss: {total_loss/len(unique_users)}")                
                if total_loss/len(unique_users) < self.eps:
                    break
                
            t+=1
        
        return self.W, self.H, self.B_i  
    
    def similar_items(self,item_id):
        item_id-=1
        nbrs = NearestNeighbors(n_neighbors=10).fit((self.H+self.B_i).T)
        distances, indices = nbrs.kneighbors(((self.H+self.B_i).T)[item_id].reshape(1,-1))
        return list(zip(indices[0] + 1, distances[0]))
    
    def recommend(self, user_id, M, movie_info):
        user_movie_matrix = self.W@self.H+self.B_i
        user_row = user_movie_matrix[user_id - 1]
        watched_movies = M[M['user_id'] == user_id]['movie_id']
        unwatched_movies = [i for i in movie_info['movie_id'] if i not in watched_movies]
        unwatched_movies.sort(key = lambda x: user_row[x-1], reverse = True)
        return list(zip(unwatched_movies[:10], user_row[unwatched_movies[:10]]))
    

In [53]:
%%time
model = WARP(iters = int(10))
W, H, B_i = model.fit(implicit_ratings, movie_info)

end of pre-count
step: 0, loss: 175.36033314524676
step: 1, loss: 148.03828865796763
step: 2, loss: 130.82973949673115
step: 3, loss: 115.97277255154448
step: 4, loss: 103.61518203306797
step: 5, loss: 93.92725508263634
step: 6, loss: 85.67512372177637
step: 7, loss: 79.150951913019
step: 8, loss: 73.62876231682986
step: 9, loss: 69.66579190457696
CPU times: user 15min 50s, sys: 1.07 s, total: 15min 51s
Wall time: 15min 50s


In [54]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 '589    Silence of the Lambs, The (1991)',
 '1250    Back to the Future (1985)',
 '907    Wizard of Oz, The (1939)',
 '604    Fargo (1996)',
 '1180    Raiders of the Lost Ark (1981)',
 '315    Shawshank Redemption, The (1994)',
 '33    Babe (1995)',
 '2728    Big (1988)']

In [55]:
get_recommendations(4, model)

['1196    Alien (1979)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '847    Godfather, The (1972)',
 '537    Blade Runner (1982)',
 '2502    Matrix, The (1999)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '1220    Terminator, The (1984)',
 '1366    Jaws (1975)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '1203    Godfather: Part II, The (1974)']